In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator

from functions import *

pd.set_option('display.max_columns', 100)

In [2]:
def application(df):

    # general cleaning procedures
    df = df.replace(['XNA', 'Unknown', 'not specified'], np.nan)
    # df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
    df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)

    # Flag_document features - count and kurtosis
    docs = [f for f in df.columns if 'FLAG_DOC' in f]
    df['DOCUMENT_COUNT'] = df[docs].copy().sum(axis=1)
    df['NEW_DOC_KURT'] = df[docs].copy().kurtosis(axis=1)

    def get_age_label(days_birth):
        """ Return the age group label (int). """
        age_years = -days_birth / 365
        if age_years < 27: return 1
        elif age_years < 40: return 2
        elif age_years < 50: return 3
        elif age_years < 65: return 4
        elif age_years < 99: return 5
        else: return 0

    # Categorical age - based on target=1 plot
    df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x: get_age_label(x))

    # New features based on External sources
    df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3
    np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')

    for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        df[feature_name] = eval('np.{}'.format(function_name))(
            df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

    # Credit ratios
    df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    
    # Income ratios
    df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
    df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']
    
    # Time ratios
    df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
    df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

    # EXT_SOURCE_X FEATURE
    df['APPS_EXT_SOURCE_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['APPS_EXT_SOURCE_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df['APPS_EXT_SOURCE_STD'] = df['APPS_EXT_SOURCE_STD'].fillna(df['APPS_EXT_SOURCE_STD'].mean())
    df['APP_SCORE1_TO_BIRTH_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_BIRTH'] / 365.25)
    df['APP_SCORE2_TO_BIRTH_RATIO'] = df['EXT_SOURCE_2'] / (df['DAYS_BIRTH'] / 365.25)
    df['APP_SCORE3_TO_BIRTH_RATIO'] = df['EXT_SOURCE_3'] / (df['DAYS_BIRTH'] / 365.25)
    df['APP_SCORE1_TO_EMPLOY_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_EMPLOYED'] / 365.25)
    df['APP_EXT_SOURCE_2*EXT_SOURCE_3*DAYS_BIRTH'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['DAYS_BIRTH']
    df['APP_SCORE1_TO_FAM_CNT_RATIO'] = df['EXT_SOURCE_1'] / df['CNT_FAM_MEMBERS']
    df['APP_SCORE1_TO_GOODS_RATIO'] = df['EXT_SOURCE_1'] / df['AMT_GOODS_PRICE']
    df['APP_SCORE1_TO_CREDIT_RATIO'] = df['EXT_SOURCE_1'] / df['AMT_CREDIT']
    df['APP_SCORE1_TO_SCORE2_RATIO'] = df['EXT_SOURCE_1'] / df['EXT_SOURCE_2']
    df['APP_SCORE1_TO_SCORE3_RATIO'] = df['EXT_SOURCE_1'] / df['EXT_SOURCE_3']
    df['APP_SCORE2_TO_CREDIT_RATIO'] = df['EXT_SOURCE_2'] / df['AMT_CREDIT']
    df['APP_SCORE2_TO_REGION_RATING_RATIO'] = df['EXT_SOURCE_2'] / df['REGION_RATING_CLIENT']
    df['APP_SCORE2_TO_CITY_RATING_RATIO'] = df['EXT_SOURCE_2'] / df['REGION_RATING_CLIENT_W_CITY']
    df['APP_SCORE2_TO_POP_RATIO'] = df['EXT_SOURCE_2'] / df['REGION_POPULATION_RELATIVE']
    df['APP_SCORE2_TO_PHONE_CHANGE_RATIO'] = df['EXT_SOURCE_2'] / df['DAYS_LAST_PHONE_CHANGE']
    df['APP_EXT_SOURCE_1*EXT_SOURCE_2'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']
    df['APP_EXT_SOURCE_1*EXT_SOURCE_3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']
    df['APP_EXT_SOURCE_1*DAYS_EMPLOYED'] = df['EXT_SOURCE_1'] * df['DAYS_EMPLOYED']
    df['APP_EXT_SOURCE_2*EXT_SOURCE_3'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['APP_EXT_SOURCE_2*DAYS_EMPLOYED'] = df['EXT_SOURCE_2'] * df['DAYS_EMPLOYED']
    df['APP_EXT_SOURCE_3*DAYS_EMPLOYED'] = df['EXT_SOURCE_3'] * df['DAYS_EMPLOYED']

    # AMT_INCOME_TOTAL : income
    # CNT_FAM_MEMBERS  : the number of family members
    df['APPS_GOODS_INCOME_RATIO'] = df['AMT_GOODS_PRICE'] / df['AMT_INCOME_TOTAL']
    df['APPS_CNT_FAM_INCOME_RATIO'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH : Client's age in days at the time of application
    # DAYS_EMPLOYED : How many days before the application the person started current employment
    df['APPS_INCOME_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']

    # other feature from better than 0.8
    df['CREDIT_TO_GOODS_RATIO_2'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio'] = df['AMT_INCOME_TOTAL'] / 12. - df['AMT_ANNUITY']
    df['APP_INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
    df['APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    df['APP_DAYS_EMPLOYED_DAYS_BIRTH_diff'] = df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']

    print('"Application_Train_Test" final shape:', df.shape)
    return df

raw

In [3]:
app_train = pd.read_csv('raw-data/dseb63_application_train.csv')
app_train.drop(['Unnamed: 0'], axis=1, inplace=True)
app_train.set_index('SK_ID_CURR', inplace=True)
app_train.head(10)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,...,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
278621,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,...,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
139008,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
138348,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,Religion,NaN,0.322738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
64140,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,0.035792,-16941,-1588,-4970.0,-477,NaN,1,1,1,1,1,0,Laborers,2.0,2,2,WEDNESDAY,16,0,0,0,0,0,0,Other,NaN,0.354225,0.621226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2536.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
219374,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,1530000.0,Unaccompanied,State servant,Higher education,Married,

In [4]:
app_train.isnull().sum()

TARGET                            0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
FLAG_OWN_REALTY                   0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     33211
AMT_REQ_CREDIT_BUREAU_WEEK    33211
AMT_REQ_CREDIT_BUREAU_MON     33211
AMT_REQ_CREDIT_BUREAU_QRT     33211
AMT_REQ_CREDIT_BUREAU_YEAR    33211
Length: 121, dtype: int64

process

In [5]:
# idenitfy col with missing values
missing_value_cols = app_train.columns[app_train.isnull().sum() > 0].tolist()

indicator = MissingIndicator()
mask_missing_values_only = indicator.fit_transform(app_train[missing_value_cols])
new_cols = [c+'_missing_flag' for c in missing_value_cols]
mask_missing_values_only = pd.DataFrame(mask_missing_values_only, columns=new_cols, index=app_train.index)
mask_missing_values_only.shape

(246009, 67)

In [6]:
# concat the missing value indicator with the original df
app_train = pd.merge(app_train, mask_missing_values_only, left_index=True, right_index=True)
app_train.head(10)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,...,FLOORSMIN_AVG_missing_flag,LANDAREA_AVG_missing_flag,LIVINGAPARTMENTS_AVG_missing_flag,LIVINGAREA_AVG_missing_flag,NONLIVINGAPARTMENTS_AVG_missing_flag,NONLIVINGAREA_AVG_missing_flag,APARTMENTS_MODE_missing_flag,BASEMENTAREA_MODE_missing_flag,YEARS_BEGINEXPLUATATION_MODE_missing_flag,YEARS_BUILD_MODE_missing_flag,COMMONAREA_MODE_missing_flag,ELEVATORS_MODE_missing_flag,ENTRANCES_MODE_missing_flag,FLOORSMAX_MODE_missing_flag,FLOORSMIN_MODE_missing_flag,LANDAREA_MODE_missing_flag,LIVINGAPARTMENTS_MODE_missing_flag,LIVINGAREA_MODE_missing_flag,NONLIVINGAPARTMENTS_MODE_missing_flag,NONLIVINGAREA_MODE_missing_flag,APARTMENTS_MEDI_missing_flag,BASEMENTAREA_MEDI_missing_flag,YEARS_BEGINEXPLUATATION_MEDI_missing_flag,YEARS_BUILD_MEDI_missing_flag,COMMONAREA_MEDI_missing_flag,ELEVATORS_MEDI_missing_flag,ENTRANCES_MEDI_missing_flag,FLOORSMAX_MEDI_missing_flag,FLOORSMIN_MEDI_missing_flag,LANDAREA_MEDI_missing_flag,LIVINGAPARTMENTS_MEDI_missing_flag,LIVINGAREA_MEDI_missing_flag,NONLIVINGAPARTMENTS_MEDI_missing_flag,NONLIVINGAREA_MEDI_missing_flag,FONDKAPREMONT_MODE_missing_flag,HOUSETYPE_MODE_missing_flag,TOTALAREA_MODE_missing_flag,WALLSMATERIAL_MODE_missing_flag,EMERGENCYSTATE_MODE_missing_flag,OBS_30_CNT_SOCIAL_CIRCLE_missing_flag,DEF_30_CNT_SOCIAL_CIRCLE_missing_flag,OBS_60_CNT_SOCIAL_CIRCLE_missing_flag,DEF_60_CNT_SOCIAL_CIRCLE_missing_flag,DAYS_LAST_PHONE_CHANGE_missing_flag,AMT_REQ_CREDIT_BUREAU_HOUR_missing_flag,AMT_REQ_CREDIT_BUREAU_DAY_missing_flag,AMT_REQ_CREDIT_BUREAU_WEEK_missing_flag,AMT_REQ_CREDIT_BUREAU_MON_missing_flag,AMT_REQ_CREDIT_BUREAU_QRT_missing_flag,AMT_REQ_CREDIT_BUREAU_YEAR_missing_flag
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
278621,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
139008,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,True,True,True,True,True,True
138348,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.

In [7]:
app_train['TARGET'].isnull().sum()

0

In [8]:
app_train_clean = application(app_train)
app_train_clean.head(20)

"Application_Train_Test" final shape: (246009, 371)


,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,...,EXT_SOURCES_PROD,EXT_SOURCES_WEIGHTED,EXT_SOURCES_MIN,EXT_SOURCES_MAX,EXT_SOURCES_MEAN,EXT_SOURCES_NANMEDIAN,EXT_SOURCES_VAR,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,CREDIT_TO_GOODS_RATIO,INCOME_TO_EMPLOYED_RATIO,INCOME_TO_BIRTH_RATIO,ID_TO_BIRTH_RATIO,CAR_TO_BIRTH_RATIO,CAR_TO_EMPLOYED_RATIO,PHONE_TO_BIRTH_RATIO,APPS_EXT_SOURCE_MEAN,APPS_EXT_SOURCE_STD,APP_SCORE1_TO_BIRTH_RATIO,APP_SCORE2_TO_BIRTH_RATIO,APP_SCORE3_TO_BIRTH_RATIO,APP_SCORE1_TO_EMPLOY_RATIO,APP_EXT_SOURCE_2*EXT_SOURCE_3*DAYS_BIRTH,APP_SCORE1_TO_FAM_CNT_RATIO,APP_SCORE1_TO_GOODS_RATIO,APP_SCORE1_TO_CREDIT_RATIO,APP_SCORE1_TO_SCORE2_RATIO,APP_SCORE1_TO_SCORE3_RATIO,APP_SCORE2_TO_CREDIT_RATIO,APP_SCORE2_TO_REGION_RATING_RATIO,APP_SCORE2_TO_CITY_RATING_RATIO,APP_SCORE2_TO_POP_RATIO,APP_SCORE2_TO_PHONE_CHANGE_RATIO,APP_EXT_SOURCE_1*EXT_SOURCE_2,APP_EXT_SOURCE_1*EXT_SOURCE_3,APP_EXT_SOURCE_1*DAYS_EMPLOYED,APP_EXT_SOURCE_2*EXT_SOURCE_3,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
278621,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,-1186.0,-291,NaN,1,1,0,1,1,0,2.0,1,1,11,0,0,0,0,0,0,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,...,NaN,NaN,0.311267,0.622246,0.466757,0.466757,0.024177,0.070862,0.208736,135000.0,0.132217,0.027598,1.145199,-227.272727,-16.104981,0.017358,NaN,NaN,0.049389,0.466757,0.219895,-0.006781,-0.013557,NaN,-0.095699,-3247.125160,0.155634,2.755797e-07,2.406391e-07,0.500232,NaN,4.810549e-07,0.622246,0.622246,175.726003,-0.000752,0.193685,NaN,-369.785566,NaN,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
139008,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,-9833.0,-2437,NaN,1,1,0,1,0,0,2.0,2,2,17,0,0,0,0,0,0,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.650442,0.650442,0.650442,0.650442,0.000000,0.159905,0.431748,67500.0,0.219900,0.094941,1.052803,-44.422507,-7.103394,0.128229,NaN,NaN,0.032465,0.650442,0.151116,NaN,-0.012501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.080199e-06,0.325221,0.325221,81.112569,-0.001054,NaN,NaN,NaN,NaN,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
138348,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,-4311.0,-3458,NaN,1,1,0,1,0,0,1.0,2,2,11,0,0,0,0,1,1,NaN,0.322738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.322738,0.322738,0.322738,0.322738,0.000000,0.152418,0.236842,121500.0,0.179963,0.042623,1.000000,-39.993417,-6.095725,0.173490,NaN,NaN,0.055489,0.322738,0.151116,NaN,-0.005914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.291195e-07,0.161369,0.161369,11.

In [9]:
# drop columns with more than 50% missing values
cols_missing_50 = app_train_clean.columns[app_train_clean.isnull().mean() > 0.5].tolist()
app_train_clean.drop(cols_missing_50, axis=1, inplace=True)
app_train_clean.isnull().sum().sort_values(ascending=False) / len(app_train_clean)

FLOORSMAX_AVG                            0.498083
FLOORSMAX_MODE                           0.498083
FLOORSMAX_MEDI                           0.498083
YEARS_BEGINEXPLUATATION_AVG              0.488120
YEARS_BEGINEXPLUATATION_MODE             0.488120
                                           ...   
FONDKAPREMONT_MODE_missing_flag          0.000000
NONLIVINGAREA_MEDI_missing_flag          0.000000
NONLIVINGAPARTMENTS_MEDI_missing_flag    0.000000
LIVINGAREA_MEDI_missing_flag             0.000000
NAME_EDUCATION_TYPE_Incomplete higher    0.000000
Length: 318, dtype: float64

In [10]:
# feature selection
# drop columns with variance less than 0.01
selected_features = app_train_clean.columns[app_train_clean.var() > 0.01].tolist()
app_train_clean = app_train_clean[selected_features]
app_train_clean.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,FLOORSMAX_AVG,FLOORSMAX_MODE,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,OWN_CAR_AGE_missing_flag,OCCUPATION_TYPE_missing_flag,EXT_SOURCE_1_missing_flag,...,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_nan,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_nan,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_nan,DOCUMENT_COUNT,NEW_DOC_KURT,AGE_RANGE,EXT_SOURCES_MIN,EXT_SOURCES_MAX,EXT_SOURCES_MEAN,EXT_SOURCES_NANMEDIAN,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,CREDIT_TO_GOODS_RATIO,INCOME_TO_BIRTH_RATIO,ID_TO_BIRTH_RATIO,APPS_EXT_SOURCE_MEAN,APP_SCORE2_TO_REGION_RATING_RATIO,APP_SCORE2_TO_CITY_RATING_RATIO,APP_SCORE2_TO_POP_RATIO,APP_EXT_SOURCE_2*EXT_SOURCE_3,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
278621,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,-16765,-1188.0,-1186.0,-291,1,0,1,0,2.0,1,1,11,0,0,0,0,0,0,0.622246,NaN,0.2917,0.2917,0.2917,0.0714,1.0,0.0,1.0,0.0,-828.0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,True,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,True,False,1,20.0,3,0.311267,0.622246,0.466757,0.466757,0.070862,0.208736,135000.0,1.145199,-16.104981,0.017358,0.466757,0.622246,0.622246,175.726003,NaN,-739.227981,NaN,4.183333,135000.0,1.145199,-13198.5,15577.0
139008,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,-19005,-3039.0,-9833.0,-2437,1,0,0,0,2.0,2,2,17,0,0,0,0,0,0,0.650442,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,True,1,20.0,4,0.650442,0.650442,0.650442,0.650442,0.159905,0.431748,67500.0,1.052803,-7.103394,0.128229,0.650442,0.325221,0.325221,81.112569,NaN,-1976.692297,NaN,2.200000,67500.0,1.052803,-18436.5,15966.0
138348,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,-19932,-3038.0,-4311.0,-3458,1,0,0,0,1.0,2,2,11,0,0,0,0,1,1,0.322738,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,True,1,20.0,4,0.322738,0.322738,0.322738,0.322738,0.152418,0.236842,121500.0,1.000000,-6.095725,0.173490,0.322738,0.161369,0.161369,11.25975

In [11]:
app_train_clean.to_csv('processed-data/processed_application_train.csv')

# train

In [12]:
X, y = app_train_clean.drop(['TARGET'], axis=1), app_train_clean['TARGET']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=64)

# replace inf values with nan
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_valid = X_valid.replace([np.inf, -np.inf], np.nan)

# robust scaler
robust_scaler = RobustScaler(quantile_range=(0, 98))
robust_scaler.fit(X_train)
X_train = robust_scaler.transform(X_train)
X_valid = robust_scaler.transform(X_valid)

# scale values
minmax_scaler = MinMaxScaler(feature_range=(0, 1))
minmax_scaler.fit(X_train)
X_train = minmax_scaler.transform(X_train)
X_valid = minmax_scaler.transform(X_valid)

In [13]:
# fill missing values with median
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_valid = imputer.transform(X_valid)

In [14]:
# train model
logreg = LogisticRegression(class_weight='balanced', solver='newton-cholesky')
logreg.fit(X_train, y_train)

# predict
y_pred = logreg.predict_proba(X_valid)[:, 1]

# evaluate
roc = roc_auc_score(y_valid, y_pred)
print('ROC AUC: {:.4f}'.format(roc))
print('GINI: {:.4f}'.format(2*roc - 1))

ROC AUC: 0.7464
GINI: 0.4928


# test

In [15]:
app_test = pd.read_csv('raw-data/dseb63_application_test.csv')
app_test.drop(['Unnamed: 0'], axis=1, inplace=True)
app_test.set_index('SK_ID_CURR', inplace=True)
app_test.head(10)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,...,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
83659,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.009630,-13297,-762,-637.0,-4307,19.0,1,1,0,1,0,0,Sales staff,4.0,2,2,THURSDAY,11,0,0,0,0,1,1,Business Entity Type 3,0.675878,0.604894,0.000527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
174814,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,Unaccompanied,Commercial associate,Higher education,Single / not married,House / apartment,0.006852,-14778,-1141,-1610.0,-4546,11.0,1,1,0,1,0,1,Managers,1.0,3,3,THURSDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.430827,0.425351,0.712155,0.0753,0.0568,0.9970,0.9592,0.1326,0.08,0.0517,0.4167,...,0.0760,0.0568,0.9970,0.9597,0.1335,0.08,0.0517,0.4167,0.2917,0.0748,0.0611,0.0859,0.0058,0.1142,reg oper account,block of flats,0.0754,Monolithic,No,2.0,0.0,2.0,0.0,-1071.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
179486,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-17907,-639,-2507.0,-1461,4.0,1,1,1,1,0,0,Sales staff,2.0,2,2,TUESDAY,16,0,0,0,0,0,0,Self-employed,0.527239,0.531760,0.207964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,5.0,0.0,-1435.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
57038,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,Unaccompanied,State servant,Secondary / secondary special,Widow,House / apartment,0.046220,-19626,-6982,-11167.0,-3158,NaN,1,1,0,1,0,0,High skill tech staff,1.0,1,1,FRIDAY,14,0,0,0,0,0,0,Business Entity Type 3,NaN,0.693521,0.614414,0.1320,0.0645,0.9846,NaN,NaN,0.16,0.0690,0.6250,...,0.1332,0.0645,0.9846,NaN,NaN,0.16,0.0690,0.6250,NaN,NaN,NaN,0.1657,NaN,0.0022,NaN,NaN,0.1285,Panel,No,0.0,0.0,0.0,0.0,-2000.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0

In [16]:
# Update missing_value_cols to include only those columns that have missing values
new_no_missing_values_cols = []

for col in missing_value_cols:
    if app_test[col].isnull().sum() == 0:
        missing_value_cols.remove(col)
        new_no_missing_values_cols.append(col)

new_cols = [c+'_missing_flag' for c in missing_value_cols]
mask_missing_values_only = indicator.fit_transform(app_test[missing_value_cols])        
mask_missing_values_only = pd.DataFrame(mask_missing_values_only, columns=new_cols, index=app_test.index)

# add no missing value cols to the df with all False values
for col in new_no_missing_values_cols:
    mask_missing_values_only[col+'_missing_flag'] = False

mask_missing_values_only.shape

(61502, 67)

In [17]:
app_test = pd.merge(app_test, mask_missing_values_only, left_index=True, right_index=True)
app_test.head(10)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,...,FLOORSMIN_AVG_missing_flag,LANDAREA_AVG_missing_flag,LIVINGAPARTMENTS_AVG_missing_flag,LIVINGAREA_AVG_missing_flag,NONLIVINGAPARTMENTS_AVG_missing_flag,NONLIVINGAREA_AVG_missing_flag,APARTMENTS_MODE_missing_flag,BASEMENTAREA_MODE_missing_flag,YEARS_BEGINEXPLUATATION_MODE_missing_flag,YEARS_BUILD_MODE_missing_flag,COMMONAREA_MODE_missing_flag,ELEVATORS_MODE_missing_flag,ENTRANCES_MODE_missing_flag,FLOORSMAX_MODE_missing_flag,FLOORSMIN_MODE_missing_flag,LANDAREA_MODE_missing_flag,LIVINGAPARTMENTS_MODE_missing_flag,LIVINGAREA_MODE_missing_flag,NONLIVINGAPARTMENTS_MODE_missing_flag,NONLIVINGAREA_MODE_missing_flag,APARTMENTS_MEDI_missing_flag,BASEMENTAREA_MEDI_missing_flag,YEARS_BEGINEXPLUATATION_MEDI_missing_flag,YEARS_BUILD_MEDI_missing_flag,COMMONAREA_MEDI_missing_flag,ELEVATORS_MEDI_missing_flag,ENTRANCES_MEDI_missing_flag,FLOORSMAX_MEDI_missing_flag,FLOORSMIN_MEDI_missing_flag,LANDAREA_MEDI_missing_flag,LIVINGAPARTMENTS_MEDI_missing_flag,LIVINGAREA_MEDI_missing_flag,NONLIVINGAPARTMENTS_MEDI_missing_flag,NONLIVINGAREA_MEDI_missing_flag,FONDKAPREMONT_MODE_missing_flag,HOUSETYPE_MODE_missing_flag,TOTALAREA_MODE_missing_flag,WALLSMATERIAL_MODE_missing_flag,EMERGENCYSTATE_MODE_missing_flag,OBS_30_CNT_SOCIAL_CIRCLE_missing_flag,DEF_30_CNT_SOCIAL_CIRCLE_missing_flag,OBS_60_CNT_SOCIAL_CIRCLE_missing_flag,DEF_60_CNT_SOCIAL_CIRCLE_missing_flag,AMT_REQ_CREDIT_BUREAU_HOUR_missing_flag,AMT_REQ_CREDIT_BUREAU_DAY_missing_flag,AMT_REQ_CREDIT_BUREAU_WEEK_missing_flag,AMT_REQ_CREDIT_BUREAU_MON_missing_flag,AMT_REQ_CREDIT_BUREAU_QRT_missing_flag,AMT_REQ_CREDIT_BUREAU_YEAR_missing_flag,DAYS_LAST_PHONE_CHANGE_missing_flag
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
83659,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.009630,-13297,-762,-637.0,-4307,19.0,1,1,0,1,0,0,Sales staff,4.0,2,2,THURSDAY,11,0,0,0,0,1,1,Business Entity Type 3,0.675878,0.604894,0.000527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False
174814,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,Unaccompanied,Commercial associate,Higher education,Single / not married,House / apartment,0.006852,-14778,-1141,-1610.0,-4546,11.0,1,1,0,1,0,1,Managers,1.0,3,3,THURSDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.430827,0.425351,0.712155,0.0753,0.0568,0.9970,0.9592,0.1326,0.08,0.0517,0.4167,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
179486,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apar

In [18]:
app_test_clean = application(app_test)
app_test_clean.head(20)

"Application_Train_Test" final shape: (61502, 370)


,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,...,EXT_SOURCES_PROD,EXT_SOURCES_WEIGHTED,EXT_SOURCES_MIN,EXT_SOURCES_MAX,EXT_SOURCES_MEAN,EXT_SOURCES_NANMEDIAN,EXT_SOURCES_VAR,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,CREDIT_TO_GOODS_RATIO,INCOME_TO_EMPLOYED_RATIO,INCOME_TO_BIRTH_RATIO,ID_TO_BIRTH_RATIO,CAR_TO_BIRTH_RATIO,CAR_TO_EMPLOYED_RATIO,PHONE_TO_BIRTH_RATIO,APPS_EXT_SOURCE_MEAN,APPS_EXT_SOURCE_STD,APP_SCORE1_TO_BIRTH_RATIO,APP_SCORE2_TO_BIRTH_RATIO,APP_SCORE3_TO_BIRTH_RATIO,APP_SCORE1_TO_EMPLOY_RATIO,APP_EXT_SOURCE_2*EXT_SOURCE_3*DAYS_BIRTH,APP_SCORE1_TO_FAM_CNT_RATIO,APP_SCORE1_TO_GOODS_RATIO,APP_SCORE1_TO_CREDIT_RATIO,APP_SCORE1_TO_SCORE2_RATIO,APP_SCORE1_TO_SCORE3_RATIO,APP_SCORE2_TO_CREDIT_RATIO,APP_SCORE2_TO_REGION_RATING_RATIO,APP_SCORE2_TO_CITY_RATING_RATIO,APP_SCORE2_TO_POP_RATIO,APP_SCORE2_TO_PHONE_CHANGE_RATIO,APP_EXT_SOURCE_1*EXT_SOURCE_2,APP_EXT_SOURCE_1*EXT_SOURCE_3,APP_EXT_SOURCE_1*DAYS_EMPLOYED,APP_EXT_SOURCE_2*EXT_SOURCE_3,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
83659,0,0,0,2,207000.0,465457.5,52641.0,418500.0,0.009630,-13297,-762.0,-637.0,-4307,19.0,1,1,0,1,0,0,4.0,2,2,11,0,0,0,0,1,1,0.675878,0.604894,0.000527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000216,1.958232,0.000527,0.675878,0.427100,0.604894,0.091822,0.057306,0.444724,51750.0,0.254304,0.113095,1.112204,-271.653543,-15.567421,0.323908,-0.001429,-0.024934,0.000150,0.427100,0.371124,-0.018565,-0.016616,-0.000014,-0.323969,-5436.276569,0.168970,1.615001e-06,1.452073e-06,1.117349,1281.855923,1.299569e-06,0.302447,0.302447,62.813535,-0.302447,0.408835,0.000356,-515.019089,0.000319,-460.929486,-0.401776,2.021739,51750.0,-271.653543,1.112204,-35391.0,-271.653543,0.002625,12535.0
174814,1,0,1,0,247500.0,1281712.5,48946.5,1179000.0,0.006852,-14778,-1141.0,-1610.0,-4546,11.0,1,1,0,1,0,1,1.0,3,3,10,0,0,0,0,0,0,0.430827,0.425351,0.712155,0.0753,0.0568,0.9970,0.9592,0.1326,0.08,0.0517,0.4167,0.2917,0.0735,0.0601,0.0844,0.0058,0.1118,0.0756,0.0566,0.9940,...,0.130504,3.423470,0.425351,0.712155,0.522778,0.430827,0.017937,0.077209,0.193101,247500.0,0.197764,0.038188,1.087118,-216.914987,-16.747868,0.307619,-0.000744,-0.009641,0.072473,0.522778,0.164029,-0.010648,-0.010513,-0.017601,-0.137914,-2708.105716,0.430827,3.654174e-07,3.361339e-07,1.012875,0.604962,3.318611e-07,0.141784,0.141784,62.076842,-0.000397,0.183253,0.306816,-491.573685,0.302916,-485.324946,-812.569032,4.763636,247500.0,-216.914987,1.087118,-28321.5,-216.914987,0.938650,13637.0
179486,1,0,0,0,202500.0,495000.0,39109.5,495000.0,0.035792,-17907,-639.0,-2507.0,-1461,4.0,1,1,1,1,0,0,2.0,2,2,16,0,0,0,0,0,0,0.527239,0.531760,0.207964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.058306,2.210131,0.207964,0.5

In [19]:
app_test_clean.isnull().sum().sort_values(ascending=False) / len(app_test_clean)

COMMONAREA_MEDI                          0.698107
COMMONAREA_AVG                           0.698107
COMMONAREA_MODE                          0.698107
CAR_TO_EMPLOYED_RATIO                    0.695343
NONLIVINGAPARTMENTS_AVG                  0.693392
                                           ...   
OBS_30_CNT_SOCIAL_CIRCLE_missing_flag    0.000000
EMERGENCYSTATE_MODE_missing_flag         0.000000
WALLSMATERIAL_MODE_missing_flag          0.000000
TOTALAREA_MODE_missing_flag              0.000000
NAME_INCOME_TYPE_Businessman             0.000000
Length: 370, dtype: float64

In [20]:
# drop columns with more than 50% missing values
app_test_clean = app_test_clean.drop(cols_missing_50, axis=1)
app_test_clean.isnull().sum().sort_values(ascending=False) / len(app_test_clean)

FLOORSMAX_AVG                            0.495707
FLOORSMAX_MODE                           0.495707
FLOORSMAX_MEDI                           0.495707
YEARS_BEGINEXPLUATATION_AVG              0.486570
YEARS_BEGINEXPLUATATION_MODE             0.486570
                                           ...   
HOUSETYPE_MODE_missing_flag              0.000000
FONDKAPREMONT_MODE_missing_flag          0.000000
NONLIVINGAREA_MEDI_missing_flag          0.000000
NONLIVINGAPARTMENTS_MEDI_missing_flag    0.000000
NAME_EDUCATION_TYPE_Incomplete higher    0.000000
Length: 317, dtype: float64

In [21]:
selected_features.remove('TARGET')
app_test_clean = app_test_clean[selected_features]
app_test_clean.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,FLOORSMAX_AVG,FLOORSMAX_MODE,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,OWN_CAR_AGE_missing_flag,OCCUPATION_TYPE_missing_flag,EXT_SOURCE_1_missing_flag,EXT_SOURCE_3_missing_flag,...,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_nan,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_nan,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_nan,DOCUMENT_COUNT,NEW_DOC_KURT,AGE_RANGE,EXT_SOURCES_MIN,EXT_SOURCES_MAX,EXT_SOURCES_MEAN,EXT_SOURCES_NANMEDIAN,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,CREDIT_TO_GOODS_RATIO,INCOME_TO_BIRTH_RATIO,ID_TO_BIRTH_RATIO,APPS_EXT_SOURCE_MEAN,APP_SCORE2_TO_REGION_RATING_RATIO,APP_SCORE2_TO_CITY_RATING_RATIO,APP_SCORE2_TO_POP_RATIO,APP_EXT_SOURCE_2*EXT_SOURCE_3,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
83659,0,0,0,2,207000.0,465457.5,52641.0,418500.0,-13297,-762.0,-637.0,-4307,1,0,0,0,4.0,2,2,11,0,0,0,0,1,1,0.604894,0.000527,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2.0,1,0,0,0,0.0,0.0,1.0,0.0,1.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,True,1,20.0,2,0.000527,0.675878,0.427100,0.604894,0.057306,0.444724,51750.0,1.112204,-15.567421,0.323908,0.427100,0.302447,0.302447,62.813535,0.000319,-460.929486,-0.401776,2.021739,51750.0,1.112204,-35391.0,12535.0
174814,1,0,1,0,247500.0,1281712.5,48946.5,1179000.0,-14778,-1141.0,-1610.0,-4546,1,0,0,1,1.0,3,3,10,0,0,0,0,0,0,0.425351,0.712155,0.4167,0.3333,0.4167,0.0754,2.0,0.0,2.0,0.0,-1071.0,1,0,0,0,0.0,0.0,1.0,0.0,3.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,1,20.0,3,0.425351,0.712155,0.522778,0.430827,0.077209,0.193101,247500.0,1.087118,-16.747868,0.307619,0.522778,0.141784,0.141784,62.076842,0.302916,-485.324946,-812.569032,4.763636,247500.0,1.087118,-28321.5,13637.0
179486,1,0,0,0,202500.0,495000.0,39109.5,495000.0,-17907,-639.0,-2507.0,-1461,1,1,0,0,2.0,2,2,16,0,0,0,0,0,0,0.531760,0.207964,NaN,NaN,NaN,NaN,5.0,0.0,5.0,0.0,-1435.0,1,0,0,0,0.0,0.0,1.0,0.0,3.0,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,True,1,20.0,3,0.207964,0.531760,0.422321,0.527239,0.035684,0.409091,1

In [22]:
# reorder columns to match train
app_test_clean = app_test_clean[X.columns]


app_test_clean.to_csv('processed-data/processed_application_test.csv')

In [23]:
# replace inf values with nan
X_test = app_test_clean.replace([np.inf, -np.inf], np.nan)

# robust scaler
X_test = robust_scaler.transform(X_test)

# scale values
X_test = minmax_scaler.transform(X_test)

# fill missing values with median
X_test = imputer.transform(X_test)

# predict
y_pred = logreg.predict_proba(X_test)[:, 1]

# submission
submission = pd.DataFrame({'SK_ID_CURR': app_test_clean.index, 'TARGET': y_pred})

In [24]:
submission.set_index('SK_ID_CURR', inplace=True)
submission.sort_index(inplace=True)
submission.head(10)

,TARGET
SK_ID_CURR,
5,0.375056
13,0.756382
16,0.492746
19,0.823665
25,0.509694
30,0.244765
43,0.158083
44,0.508947
53,0.438456


In [25]:
submission.to_csv('submission.csv')